In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
from math import ceil

from simulation import run_simulation
from graph_create import ring_lattice, reduce_providers_simplest, powerlaw_dist_time
import util

init_notebook_mode(connected=True)

In [2]:
# Constants
num_vertices = 100
regularity = 4

In [3]:
def plot_conv(conv_rates, r, diam, show_model=False):
    global regularity
    
    fig = go.Figure()
    
    normalize_cr = lambda cr : cr / (r / 2)

    #Add averages trace
    prk_vals = [ p * r / regularity for (p, cr) in conv_rates ]
    cr_vals = [ normalize_cr(np.average(cr)) for (p, cr) in conv_rates ]
    fig.add_trace(go.Scatter(x=prk_vals,
        y=cr_vals, mode='lines+markers', hoverinfo='skip',
        name='avg conv rate'))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_cr_vals = []
    for (p, cr) in conv_rates:
        indiv_cr_vals.extend(normalize_cr(cr))
        indiv_p_vals.extend([p * r / regularity] * len(cr))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_cr_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='conv rates per c'))
    
    #Add model trace
    if show_model:
        p_vals = [ p for (p, cr) in conv_rates ]
        expec_cr = [ normalize_cr(util.expected_conv_rate_simp(p, r, regularity, diam)) for p in p_vals ]
        fig.add_trace(go.Scatter(x=prk_vals,
            y=expec_cr, mode='lines+markers', hoverinfo='skip',
            name='expected conv rate'))
    
    fig.layout.update(showlegend=False)
    title_fmt = "Average Convergance Rate (Normalized) vs p*r/k (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p*r/k'), yaxis=dict(title='avg conv rate'))

    iplot(fig)

In [4]:
graph_diam = None
for r in np.linspace(100, 0, 10, endpoint=False):
    all_conv_rates = []
    for p in np.linspace(1, 0, 10, endpoint=False):
        conv_rates = []
        num_iter = 10
        for i in range(num_iter):
            ring_lat = ring_lattice(regularity, num_vertices, p, r)
            
            if not graph_diam:
                graph_diam = util.calc_diameter(ring_lat)
                
            simp_ring_lat = reduce_providers_simplest(ring_lat)

            sim_g, sim_utils = run_simulation(ring_lat)
            
            if sum(sim_utils[-1]) != len(sim_utils[-1]):
                conv_rates.append(-1 * len(sim_utils))
            else:
                conv_rates.append(len(sim_utils))
        all_conv_rates.append((p, conv_rates))
        
    plot_conv(all_conv_rates, r, graph_diam, show_model=True)

In [5]:
graph_diam = None
for r in np.linspace(100, 0, 10, endpoint=False):
    all_conv_rates = []
    for p in np.linspace(1, 0, 10, endpoint=False):
        conv_rates = []
        num_iter = 10
        for i in range(num_iter):
            ring_lat = ring_lattice(regularity, num_vertices, p, r)
            
            if not graph_diam:
                graph_diam = util.calc_diameter(ring_lat)
                
            simp_ring_lat = reduce_providers_simplest(ring_lat)
            
            powerlaw_dist_time(ring_lat, 2)

            sim_g, sim_utils = run_simulation(ring_lat)
            
            if sum(sim_utils[-1]) != len(sim_utils[-1]):
                conv_rates.append(-1 * len(sim_utils))
            else:
                conv_rates.append(len(sim_utils))
        all_conv_rates.append((p, conv_rates))
        
    plot_conv(all_conv_rates, r, graph_diam)